In [151]:
using PyPlot, PyCall
using JuMP, Ipopt, Statistics
inset_locator = pyimport("mpl_toolkits.axes_grid.inset_locator");
mpl = pyimport("matplotlib");

In [152]:
include("../src/model.jl");
include("../src/diagnostics.jl");
include("../src/optimization.jl");
include("../src/plotting.jl");
include("../src/defaults.jl");

### Make figures from default model configuration for manuscript

#### Approach 1: Cost-benefit analysis

In [153]:
model = ClimateModel();
@time optimize_controls!(model, obj_option = "net_benefit");

Solve_Succeeded
  3.297077 seconds (3.98 M allocations: 198.542 MiB, 1.56% gc time)


In [ ]:
stat_dict = Dict("mitigate" => [], "remove" => [], "geoeng" => [], "adapt" => [], "maxTnoG" => [], "maxT" => [], "maxCO2" => [])
ρdict = Dict()
ρs = 0.:0.005:0.1

for ρ in ρs
    tmp_model = ClimateModel();
    tmp_model.economics.utility_discount_rate = ρ
    @time optimize_controls!(tmp_model, obj_option = "net_benefit");
    ρdict[ρ] = tmp_model
    
    append!(stat_dict["mitigate"], mean(tmp_model.controls.mitigate))
    append!(stat_dict["remove"], mean(tmp_model.controls.remove))
    append!(stat_dict["adapt"], mean(tmp_model.controls.adapt))
    append!(stat_dict["geoeng"], mean(tmp_model.controls.geoeng))
    append!(stat_dict["maxTnoG"], maximum(δT_no_geoeng(tmp_model)))
    append!(stat_dict["maxT"], maximum(δT(tmp_model)))
    append!(stat_dict["maxCO2"], maximum(CO₂(tmp_model)))
end

Solve_Succeeded
  0.046425 seconds (113.94 k allocations: 4.394 MiB)
Solve_Succeeded
  0.080304 seconds (113.94 k allocations: 4.394 MiB, 39.93% gc time)
Solve_Succeeded
  0.043396 seconds (110.14 k allocations: 4.327 MiB)
Solve_Succeeded
  0.046371 seconds (106.05 k allocations: 4.241 MiB)
Solve_Succeeded
  0.040423 seconds (102.11 k allocations: 4.166 MiB)
Solve_Succeeded
  0.038739 seconds (100.90 k allocations: 4.143 MiB)
Solve_Succeeded
  0.041837 seconds (100.76 k allocations: 4.133 MiB)
Solve_Succeeded
  0.040541 seconds (98.16 k allocations: 4.089 MiB)
Solve_Succeeded
  0.039657 seconds (98.31 k allocations: 4.099 MiB)
Solve_Succeeded
  0.041920 seconds (98.16 k allocations: 4.089 MiB)
Solve_Succeeded
  0.046127 seconds (106.05 k allocations: 4.242 MiB)
Solve_Succeeded
  0.065610 seconds (118.03 k allocations: 4.479 MiB)
Solve_Succeeded
  0.070700 seconds (117.88 k allocations: 4.470 MiB)
Solve_Succeeded
  0.831575 seconds (1.08 M allocations: 22.942 MiB)
Solve_Succeeded
  1.96

In [ ]:
fig = figure(figsize=(12.75,3.75))

fig.add_subplot(1,3,1)
plot_controls(model)
legend(handlelength=1.5, loc="upper right")
add_label("a)")
xticks(model.domain[1]:40.:2200.)

ax = fig.add_subplot(1,3,2)
plot_benefits(model)
legend(handlelength=1.5, loc="upper left")
add_label("b)")
ylim([-3., 25.])
xticks(model.domain[1]:40.:2200.)

ax = fig.add_subplot(1,3,3)
plot(ρs, stat_dict["mitigate"], label=L"$\overline{M}$")
plot(ρs, stat_dict["remove"], label=L"$\overline{R}$")
plot(ρs, stat_dict["adapt"], label=L"$\overline{A}$")
plot(ρs, stat_dict["geoeng"], label=L"$\overline{G}$")
legend(ncol=2, handlelength=1.)
ylim([0, 0.6])
xlim([0., 0.1])
xticks([0., 0.025, 0.05, 0.075, 0.1], ["0%", "2.5%", "5%", "7.5%", "10%"])
grid(true)
ylabel("fractional control deployment")
title("control preferences vs. discount rate")
title("c)", loc="left")
xlabel("discount rate")

iax = fig.add_axes([0.535, 0.28, 0.11, 0.155])
sca(iax)
plot_benefits(model)
ylabel("")
xlabel("")
ylim([-1.5, 1.5])
xlim([2020,2070])
title("")

tight_layout()
savefig("../figures/default-benefits_controls_and_benefits.pdf", dpi=150, bbox_inches="tight")

print("Maximum warming: ", maximum(δT_adapt(model)))

#### Approach 2: Avoiding damage thresholds

In [ ]:
model = ClimateModel();
@time optimize_controls!(model);

In [ ]:
figure(figsize=(12.5,3.75))

subplot(1,3,1)
plot_emissions(model)
legend()
add_label("a)")
subplot(1,3,2)
plot_concentrations(model)
legend()
add_label("b)")
subplot(1,3,3)
plot_temperatures(model)
legend(loc="upper left")
add_label("c)")
tight_layout()
savefig("../figures/default-temp_carbon_and_temperatures.pdf", dpi=150, bbox_inches="tight")

In [ ]:
function f_asymp(α, μ)
    return α .^ 2. ./ (1. .- exp.(-(1 .- α)/(1. - μ)))
end
α = 0.:0.002:1.0

In [ ]:
tmp_model = ClimateModel();
@time m = optimize_controls!(
    tmp_model,
    mitigation_penetration = μ,
);


In [ ]:
figure(figsize=(9.5,4.))

subplot(1,2,1)
plot_controls(model)
legend(loc="upper right")
title("a)", loc="left")

subplot(1,2,2)
plot_damages(model)
legend(loc="upper right")
title("b)", loc="left")
ylim([0, 20])

tight_layout()
savefig("../figures/default-temp_controls_and_damages.pdf", dpi=150, bbox_inches="tight")

print("Net Present Control Costs: ", discounted_total_control_cost(model))